In [23]:
import math
import pandas as pd
import numpy as np
from scipy.stats import norm

In [48]:
stockprice = 4072.43
t = 3
rfr = .0268
dividendyield = .0155
correlation= .265 
volatilityfx= .073
vol = .01
sims = 10000
floor = 0.07
participation = 1
ceiling = .15

In [49]:
dffinal = pd.DataFrame(columns=['trial','randomshock','stock','antithetic','return','antitheticreturn','effectivereturn','antieffectivereturn'])
for i in range(0,sims):
    randshock = np.random.uniform(low=0, high=1)
    stockp = stockprice*math.exp(((rfr-(dividendyield+correlation*volatilityfx*vol)-(vol**2)/2)*t)+vol*norm.ppf(randshock)*math.sqrt(t))
    antistockp = stockprice*math.exp(((rfr-(dividendyield+correlation*volatilityfx*vol)-(vol**2)/2)*t)-vol*norm.ppf(randshock)*math.sqrt(t))
    return1 = np.log(stockp/stockprice)
    antitheticreturn = np.log(antistockp/stockprice)
    dffinal = dffinal.append({'trial':i+1,
                    'randomshock':randshock,
                    'stock':stockp,
                    'antithetic':antistockp,
                    'return': return1,
                    'antitheticreturn': antitheticreturn,
                    'effectivereturn': max(min(return1*participation,ceiling),floor),
                    'antieffectivereturn':max(min(antitheticreturn*participation,ceiling),floor) },ignore_index=True)



In [50]:
dffinal['pvpayoff']= 1000*(1+dffinal['effectivereturn'])*math.exp(-rfr*t)
dffinal['pvantitheticpayoff']= 1000*(1+dffinal['antieffectivereturn'])*math.exp(-rfr*t)
dffinal['Averagepayoff']= (np.mean(dffinal['pvpayoff']) + np.mean(dffinal['pvantitheticpayoff']))/2

In [51]:
dffinal

,trial,randomshock,stock,antithetic,return,antitheticreturn,effectivereturn,antieffectivereturn,pvpayoff,pvantitheticpayoff,Averagepayoff
0,1.0,0.999488,4456.172276,3977.004439,0.090050,-0.023711,0.09005,0.07,1005.840825,987.339476,987.448349
1,2.0,0.088196,4112.343454,4309.517704,0.009753,0.056586,0.07000,0.07,987.339476,987.339476,987.448349
2,3.0,0.348880,4181.554851,4238.188318,0.026443,0.039896,0.07000,0.07,987.339476,987.339476,987.448349
3,4.0,0.968395,4347.434851,4076.476711,0.065346,0.000993,0.07000,0.07,987.339476,987.339476,987.448349
4,5.0,0.199685,4148.773375,4271.676305,0.018573,0.047766,0.07000,0.07,987.339476,987.339476,987.448349
...,...,...,...,...,...,...,...,...,...,...,...
9995,9996.0,0.827654,4279.243706,4141.436697,0.049536,0.016803,0.07000,0.07,987.339476,987.339476,987.448349
9996,9997.0,0.188559,4145.866248,4274.671652,0.017872,0.048467,0.07000,0.07,987.339476,987.339476,987.448349
9997,9998.0,0.026041,4070.497953,4353.820375,-0.000475,0.066814,0.07000,0.07,987.339476,987.339476,987.448349
9998,9999.0,0.151948,4135.474323,4285.413362,0.015362,0.050977,0.07000,0.07,987.339476,987.339476,987.448349


In [22]:
(dffinal['effectivereturn']==.15).sum()

2948